In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
from scipy import interp
from sklearn.model_selection import StratifiedKFold
import matplotlib.patches as patches
from sklearn.metrics import mean_squared_error

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
from sklearn.metrics import *
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
#from tensorflow.keras.engine.topology import Layer
from tensorflow.keras import initializers, regularizers, constraints

E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
data = pd .read_csv(r'DL_S4Pr15Data.csv')

In [3]:
data.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [4]:
print(data.columns)

Index(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label'],
      dtype='object')


In [5]:
label_value_count = data.label.value_counts()
print(label_value_count)

male      1584
female    1584
Name: label, dtype: int64


In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [7]:
data.describe()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.180907,0.057126,0.185621,0.140456,0.224765,0.084309,3.140168,36.568461,0.895127,0.408216,0.165282,0.180907,0.142807,0.036802,0.258842,0.829211,0.052647,5.047277,4.994630,0.173752
std,0.029918,0.016652,0.036360,0.048680,0.023639,0.042783,4.240529,134.928661,0.044980,0.177521,0.077203,0.029918,0.032304,0.019220,0.030077,0.525205,0.063299,3.521157,3.520039,0.119454
min,0.039363,0.018363,0.010975,0.000229,0.042946,0.014558,0.141735,2.068455,0.738651,0.036876,0.000000,0.039363,0.055565,0.009775,0.103093,0.007812,0.004883,0.007812,0.000000,0.000000
25%,0.163662,0.041954,0.169593,0.111087,0.208747,0.042560,1.649569,5.669547,0.861811,0.258041,0.118016,0.163662,0.116998,0.018223,0.253968,0.419828,0.007812,2.070312,2.044922,0.099766
50%,0.184838,0.059155,0.190032,0.140286,0.225684,0.094280,2.197101,8.318463,0.901767,0.396335,0.186599,0.184838,0.140519,0.046110,0.271186,0.765795,0.023438,4.992188,4.945312,0.139357
75%,0.199146,0.067020,0.210618,0.175939,0.243660,0.114175,2.931694,13.648905,0.928713,0.533676,0.221104,0.199146,0.169581,0.047904,0.277457,1.177166,0.070312,7.007812,6.992188,0.209183
max,0.251124,0.115273,0.261224,0.247347,0.273469,0.252225,34.725453,1309.612887,0.981997,0.842936,0.280000,0.251124,0.237636,0.204082,0.279114,2.957682,0.458984,21.867188,21.843750,0.932374


In [8]:
data.cov()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
meanfreq,0.000895,-0.000368,0.001007,0.001327,0.000524,-0.000803,-0.040893,-1.275764,-0.000809,-0.004166,0.001588,0.000895,0.000445,0.000221,0.000247,0.008433,0.000434,0.054730,0.054296,-0.000775
sd,-0.000368,0.000277,-0.000341,-0.000687,-0.000063,0.000623,0.022215,0.777957,0.000537,0.002477,-0.000680,-0.000368,-0.000251,-0.000111,-0.000065,-0.004222,-0.000377,-0.028278,-0.027901,0.000244
median,0.001007,-0.000341,0.001322,0.001372,0.000629,-0.000743,-0.039689,-1.194037,-0.000821,-0.004271,0.001902,0.001007,0.000487,0.000236,0.000275,0.008707,0.000440,0.056195,0.055755,-0.000926
Q25,0.001327,-0.000687,0.001372,0.002370,0.000549,-0.001821,-0.065949,-2.300099,-0.001419,-0.006627,0.002222,0.001327,0.000857,0.000300,0.000293,0.011950,0.000931,0.078794,0.077862,-0.000822
Q75,0.000524,-0.000063,0.000629,0.000549,0.000559,0.000010,-0.020684,-0.474872,-0.000186,-0.001587,0.000889,0.000524,0.000118,0.000117,0.000203,0.004459,-0.000036,0.027894,0.027930,-0.000611
IQR,-0.000803,0.000623,-0.000743,-0.001821,0.000010,0.001830,0.045265,1.825227,0.001233,0.005040,-0.001334,-0.000803,-0.000739,-0.000183,-0.000090,-0.007491,-0.000967,-0.050900,-0.049933,0.000211
skew,-0.040893,0.022215,-0.039689,-0.065949,-0.020684,0.045265,17.982084,559.020683,-0.037281,0.059992,-0.142365,-0.040893,-0.022968,-0.017682,-0.010313,-0.750211,-0.016537,-4.563846,-4.547309,-0.085771
kurt,-1.275764,0.777957,-1.194037,-2.300099,-0.474872,1.825227,559.020683,18205.743622,-0.774672,2.632018,-4.236781,-1.275764,-0.848046,-0.526967,-0.185331,-21.488711,-0.882384,-130.416362,-129.533978,-3.312846
sp.ent,-0.000809,0.000537,-0.000821,-0.001419,-0.000186,0.001233,-0.037281,-0.774672,0.002023,0.006918,-0.001130,-0.000809,-0.000746,-0.000264,-0.000163,-0.006935,-0.000840,-0.051355,-0.050516,0.001064
sfm,-0.004166,0.002477,-0.004271,-0.006627,-0.001587,0.005040,0.059992,2.632018,0.006918,0.031514,-0.006660,-0.004166,-0.002415,-0.001235,-0.001027,-0.039946,-0.003254,-0.272940,-0.269686,0.004485


In [9]:
data.corr()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
meanfreq,1.000000,-0.739039,0.925445,0.911416,0.740997,-0.627605,-0.322327,-0.316036,-0.601203,-0.784332,0.687715,1.000000,0.460844,0.383937,0.274004,0.536666,0.229261,0.519528,0.515570,-0.216979
sd,-0.739039,1.000000,-0.562603,-0.846931,-0.161076,0.874660,0.314597,0.346241,0.716620,0.838086,-0.529150,-0.739039,-0.466281,-0.345609,-0.129662,-0.482726,-0.357667,-0.482278,-0.475999,0.122660
median,0.925445,-0.562603,1.000000,0.774922,0.731849,-0.477352,-0.257407,-0.243382,-0.502005,-0.661690,0.677433,0.925445,0.414909,0.337602,0.251328,0.455943,0.191169,0.438919,0.435621,-0.213298
Q25,0.911416,-0.846931,0.774922,1.000000,0.477140,-0.874189,-0.319475,-0.350182,-0.648126,-0.766875,0.591277,0.911416,0.545035,0.320994,0.199841,0.467403,0.302255,0.459683,0.454394,-0.141377
Q75,0.740997,-0.161076,0.731849,0.477140,1.000000,0.009636,-0.206339,-0.148881,-0.174905,-0.378198,0.486857,0.740997,0.155091,0.258002,0.285584,0.359181,-0.023750,0.335114,0.335648,-0.216475
IQR,-0.627605,0.874660,-0.477352,-0.874189,0.009636,1.000000,0.249497,0.316185,0.640813,0.663601,-0.403764,-0.627605,-0.534462,-0.222680,-0.069588,-0.333362,-0.357037,-0.337877,-0.331563,0.041252
skew,-0.322327,0.314597,-0.257407,-0.319475,-0.206339,0.249497,1.000000,0.977020,-0.195459,0.079694,-0.434859,-0.322327,-0.167668,-0.216954,-0.080861,-0.336848,-0.061608,-0.305651,-0.304640,-0.169325
kurt,-0.316036,0.346241,-0.243382,-0.350182,-0.148881,0.316185,0.977020,1.000000,-0.127644,0.109884,-0.406722,-0.316036,-0.194560,-0.203201,-0.045667,-0.303234,-0.103313,-0.274500,-0.272729,-0.205539
sp.ent,-0.601203,0.716620,-0.502005,-0.648126,-0.174905,0.640813,-0.195459,-0.127644,1.000000,0.866411,-0.325298,-0.601203,-0.513194,-0.305826,-0.120738,-0.293562,-0.294869,-0.324253,-0.319054,0.198074
sfm,-0.784332,0.838086,-0.661690,-0.766875,-0.378198,0.663601,0.079694,0.109884,0.866411,1.000000,-0.485913,-0.784332,-0.421066,-0.362100,-0.192369,-0.428442,-0.289593,-0.436649,-0.431580,0.211477


In [10]:
data.shape

(3168, 21)

In [11]:
dict = {'label':{'male':1,'female':0}}      
data.replace(dict,inplace = True)           
x = data.loc[:, data.columns != 'label']
y = data.loc[:,'label']

In [12]:
x.head(5)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274


In [13]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [153]:
from sklearn.utils import shuffle
x, y = shuffle(x, y, random_state=1010101)

In [154]:
trainX = x[:int(len(x) * 0.7)]
trainY = y[:int(len(y) * 0.7)]
validateX = x[int(len(x) * 0.7) : int(len(x) * 0.9)]
validateY = y[int(len(y) * 0.7) : int(len(y) * 0.9)]
testX = x[int(len(x) * 0.9):]
testY = y[int(len(y) * 0.9):]

In [35]:
print (len(trainX))
print (len(validateX))
print (len(testX))

2217
634
317


In [36]:
trainX = np.array(trainX)
trainY = np.array(trainY)
validateX = np.array(validateX)
validateY = np.array(validateY)
testX = np.array(testX)
testY = np.array(testY)

In [37]:
print (trainX.shape)
print (testX.shape)
print (validateX.shape)

(2217, 20)
(317, 20)
(634, 20)


In [38]:
print (trainY.shape)
print (testY.shape)
print (validateY.shape)

(2217,)
(317,)
(634,)


In [39]:
model = Sequential()
model.add(Dense(len(trainX[0]), input_dim=len(trainX[0]), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [40]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [42]:
history = model.fit(trainX, trainY, epochs=2000, batch_size=1000, validation_data = (validateX, validateY))

Train on 2217 samples, validate on 634 samples
Epoch 1/2000
2217/2217 [==============================] - 0s 32us/sample - loss: 0.0253 - acc: 0.9707 - val_loss: 0.0183 - val_acc: 0.9779
Epoch 2/2000
2217/2217 [==============================] - 0s 24us/sample - loss: 0.0282 - acc: 0.9684 - val_loss: 0.0227 - val_acc: 0.9763
Epoch 3/2000
2217/2217 [==============================] - 0s 27us/sample - loss: 0.0326 - acc: 0.9603 - val_loss: 0.0231 - val_acc: 0.9716
Epoch 4/2000
2217/2217 [==============================] - 0s 24us/sample - loss: 0.0305 - acc: 0.9648 - val_loss: 0.0214 - val_acc: 0.9732
Epoch 5/2000
2217/2217 [==============================] - 0s 25us/sample - loss: 0.0264 - acc: 0.9716 - val_loss: 0.0217 - val_acc: 0.9732
Epoch 6/2000
2217/2217 [==============================] - 0s 31us/sample - loss: 0.0266 - acc: 0.9684 - val_loss: 0.0202 - val_acc: 0.9779
Epoch 7/2000
2217/2217 [==============================] - 0s 20us/sample - loss: 0.0257 - acc: 0.9720 - val_loss: 0.020

Epoch 59/2000
2217/2217 [==============================] - 0s 21us/sample - loss: 0.0238 - acc: 0.9734 - val_loss: 0.0202 - val_acc: 0.9763
Epoch 60/2000
2217/2217 [==============================] - 0s 18us/sample - loss: 0.0247 - acc: 0.9725 - val_loss: 0.0214 - val_acc: 0.9748
Epoch 61/2000
2217/2217 [==============================] - 0s 18us/sample - loss: 0.0255 - acc: 0.9707 - val_loss: 0.0188 - val_acc: 0.9779
Epoch 62/2000
2217/2217 [==============================] - 0s 18us/sample - loss: 0.0237 - acc: 0.9738 - val_loss: 0.0200 - val_acc: 0.9748
Epoch 63/2000
2217/2217 [==============================] - 0s 22us/sample - loss: 0.0238 - acc: 0.9720 - val_loss: 0.0195 - val_acc: 0.9763
Epoch 64/2000
2217/2217 [==============================] - 0s 33us/sample - loss: 0.0233 - acc: 0.9734 - val_loss: 0.0188 - val_acc: 0.9779
Epoch 65/2000
2217/2217 [==============================] - 0s 22us/sample - loss: 0.0246 - acc: 0.9720 - val_loss: 0.0186 - val_acc: 0.9748
Epoch 66/2000
2217/2

2217/2217 [==============================] - 0s 14us/sample - loss: 0.0242 - acc: 0.9720 - val_loss: 0.0202 - val_acc: 0.9732
Epoch 176/2000
2217/2217 [==============================] - 0s 13us/sample - loss: 0.0231 - acc: 0.9743 - val_loss: 0.0186 - val_acc: 0.9763
Epoch 177/2000
2217/2217 [==============================] - ETA: 0s - loss: 0.0246 - acc: 0.972 - 0s 9us/sample - loss: 0.0230 - acc: 0.9738 - val_loss: 0.0175 - val_acc: 0.9779
Epoch 178/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0234 - acc: 0.9725 - val_loss: 0.0170 - val_acc: 0.9811
Epoch 179/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0235 - acc: 0.9725 - val_loss: 0.0200 - val_acc: 0.9732
Epoch 180/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0237 - acc: 0.9743 - val_loss: 0.0188 - val_acc: 0.9763
Epoch 181/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0247 - acc: 0.9711 - val_loss: 0.0163 - val_acc

Epoch 233/2000
2217/2217 [==============================] - 0s 13us/sample - loss: 0.0233 - acc: 0.9738 - val_loss: 0.0199 - val_acc: 0.9748
Epoch 234/2000
2217/2217 [==============================] - 0s 14us/sample - loss: 0.0227 - acc: 0.9761 - val_loss: 0.0177 - val_acc: 0.9748
Epoch 235/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0223 - acc: 0.9747 - val_loss: 0.0178 - val_acc: 0.9779
Epoch 236/2000
2217/2217 [==============================] - 0s 13us/sample - loss: 0.0225 - acc: 0.9752 - val_loss: 0.0194 - val_acc: 0.9763
Epoch 237/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0226 - acc: 0.9743 - val_loss: 0.0183 - val_acc: 0.9795
Epoch 238/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0222 - acc: 0.9756 - val_loss: 0.0184 - val_acc: 0.9763
Epoch 239/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0233 - acc: 0.9734 - val_loss: 0.0176 - val_acc: 0.9779
Epoch 240/2000

Epoch 291/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0275 - acc: 0.9680 - val_loss: 0.0206 - val_acc: 0.9795
Epoch 292/2000
2217/2217 [==============================] - ETA: 0s - loss: 0.0255 - acc: 0.972 - 0s 17us/sample - loss: 0.0263 - acc: 0.9702 - val_loss: 0.0292 - val_acc: 0.9685
Epoch 293/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0376 - acc: 0.9549 - val_loss: 0.0228 - val_acc: 0.9700
Epoch 294/2000
2217/2217 [==============================] - ETA: 0s - loss: 0.0331 - acc: 0.965 - 0s 11us/sample - loss: 0.0279 - acc: 0.9707 - val_loss: 0.0189 - val_acc: 0.9795
Epoch 295/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0266 - acc: 0.9707 - val_loss: 0.0218 - val_acc: 0.9779
Epoch 296/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0274 - acc: 0.9648 - val_loss: 0.0196 - val_acc: 0.9795
Epoch 297/2000
2217/2217 [==============================] - 0s 11us/sample - lo

...

Epoch 1506/2000
2217/2217 [==============================] - 0s 16us/sample - loss: 0.0187 - acc: 0.9802 - val_loss: 0.0171 - val_acc: 0.9826
Epoch 1507/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0182 - acc: 0.9806 - val_loss: 0.0200 - val_acc: 0.9779
Epoch 1508/2000
2217/2217 [==============================] - 0s 18us/sample - loss: 0.0181 - acc: 0.9811 - val_loss: 0.0221 - val_acc: 0.9716
Epoch 1509/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0199 - acc: 0.9783 - val_loss: 0.0210 - val_acc: 0.9795
Epoch 1510/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0200 - acc: 0.9783 - val_loss: 0.0177 - val_acc: 0.9811
Epoch 1511/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0197 - acc: 0.9783 - val_loss: 0.0176 - val_acc: 0.9811
Epoch 1512/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0192 - acc: 0.9793 - val_loss: 0.0257 - val_acc: 0.9700
Epoch 

Epoch 1621/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0201 - acc: 0.9797 - val_loss: 0.0258 - val_acc: 0.9685
Epoch 1622/2000
2217/2217 [==============================] - 0s 13us/sample - loss: 0.0207 - acc: 0.9783 - val_loss: 0.0222 - val_acc: 0.9669
Epoch 1623/2000
2217/2217 [==============================] - 0s 22us/sample - loss: 0.0194 - acc: 0.9797 - val_loss: 0.0177 - val_acc: 0.9779
Epoch 1624/2000
2217/2217 [==============================] - 0s 15us/sample - loss: 0.0192 - acc: 0.9802 - val_loss: 0.0191 - val_acc: 0.9763
Epoch 1625/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0186 - acc: 0.9815 - val_loss: 0.0197 - val_acc: 0.9716
Epoch 1626/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0185 - acc: 0.9802 - val_loss: 0.0214 - val_acc: 0.9700
Epoch 1627/2000
2217/2217 [==============================] - 0s 16us/sample - loss: 0.0184 - acc: 0.9802 - val_loss: 0.0189 - val_acc: 0.9763
Epoch 

Epoch 1736/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0175 - acc: 0.9820 - val_loss: 0.0176 - val_acc: 0.9811
Epoch 1737/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0183 - acc: 0.9811 - val_loss: 0.0171 - val_acc: 0.9811
Epoch 1738/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0172 - acc: 0.9820 - val_loss: 0.0185 - val_acc: 0.9811
Epoch 1739/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0176 - acc: 0.9820 - val_loss: 0.0173 - val_acc: 0.9826
Epoch 1740/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0186 - acc: 0.9797 - val_loss: 0.0176 - val_acc: 0.9811
Epoch 1741/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0183 - acc: 0.9802 - val_loss: 0.0189 - val_acc: 0.9763
Epoch 1742/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0170 - acc: 0.9815 - val_loss: 0.0257 - val_acc: 0.9685
Epoch 

Epoch 1794/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0210 - acc: 0.9770 - val_loss: 0.0215 - val_acc: 0.9716
Epoch 1795/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0199 - acc: 0.9788 - val_loss: 0.0213 - val_acc: 0.9779
Epoch 1796/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0191 - acc: 0.9802 - val_loss: 0.0208 - val_acc: 0.9716
Epoch 1797/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0207 - acc: 0.9770 - val_loss: 0.0189 - val_acc: 0.9811
Epoch 1798/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0186 - acc: 0.9806 - val_loss: 0.0208 - val_acc: 0.9779
Epoch 1799/2000
2217/2217 [==============================] - 0s 11us/sample - loss: 0.0188 - acc: 0.9802 - val_loss: 0.0273 - val_acc: 0.9621
Epoch 1800/2000
2217/2217 [==============================] - 0s 12us/sample - loss: 0.0205 - acc: 0.9774 - val_loss: 0.0217 - val_acc: 0.9763
Epoch 

2217/2217 [==============================] - 0s 9us/sample - loss: 0.0220 - acc: 0.9752 - val_loss: 0.0184 - val_acc: 0.9795
Epoch 1909/2000
2217/2217 [==============================] - 0s 10us/sample - loss: 0.0219 - acc: 0.9756 - val_loss: 0.0239 - val_acc: 0.9700
Epoch 1910/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0242 - acc: 0.9698 - val_loss: 0.0212 - val_acc: 0.9700
Epoch 1911/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0196 - acc: 0.9788 - val_loss: 0.0287 - val_acc: 0.9653
Epoch 1912/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0222 - acc: 0.9738 - val_loss: 0.0180 - val_acc: 0.9795
Epoch 1913/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0201 - acc: 0.9802 - val_loss: 0.0246 - val_acc: 0.9763
Epoch 1914/2000
2217/2217 [==============================] - 0s 9us/sample - loss: 0.0242 - acc: 0.9747 - val_loss: 0.0255 - val_acc: 0.9700
Epoch 1915/2000
2217/2217 [=

In [64]:
predictions = model.predict(testX)

In [65]:
prob = []
limit_prob = []
for k in range(len(predictions)):
    prob.append(round(predictions[k][0], 4))
    if round(predictions[k][0], 4) > 0.5:
        limit_prob.append(1)
    else:
        limit_prob.append(0)

In [66]:
my_submission = pd.DataFrame({'real': testY, 'prediction_probability': prob, 'prediction_decission' : limit_prob})
# you could use any filename. We choose submission here
#my_submission.to_csv('sample_submission1.csv', index=False)

In [69]:
my_submission
a = my_submission['prediction_decission']

In [68]:
print("\nAccuracy score: %f" %(accuracy_score(testY,a) * 100))
print("Recall score : %f" %(recall_score(testY,a) * 100))
print("ROC score : %f\n" %(roc_auc_score(testY,a) * 100))
print(confusion_matrix(testY,a)) 
print("F1 score : %f\n" %(f1_score(testY,a, zero_division=1)*100))


Accuracy score: 96.845426
Recall score : 96.226415
ROC score : 96.847385

[[154   4]
 [  6 153]]
F1 score : 96.835443



In [14]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_val_predict
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=0)

In [15]:
x_train.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
943,0.200303,0.061665,0.229964,0.135451,0.256245,0.120794,1.619882,5.194025,0.886318,0.313936,0.256245,0.200303,0.127036,0.047809,0.279070,1.690770,0.023438,6.515625,6.492188,0.179513
2486,0.214223,0.037987,0.216717,0.206465,0.225556,0.019091,3.377777,15.714729,0.819595,0.217323,0.220960,0.214223,0.175808,0.047478,0.279070,2.030966,0.023438,8.859375,8.835938,0.108594
517,0.150002,0.068328,0.162544,0.093932,0.202894,0.108961,1.953073,7.469802,0.950159,0.648723,0.084131,0.150002,0.093508,0.016343,0.258065,0.470982,0.007812,4.875000,4.867188,0.135587
778,0.172459,0.059078,0.178675,0.111729,0.213958,0.102229,1.473990,5.318155,0.919637,0.360737,0.209887,0.172459,0.091120,0.016343,0.161616,0.348145,0.085938,2.882812,2.796875,0.074115
1280,0.176249,0.076477,0.189526,0.118162,0.251532,0.133370,2.258821,9.384604,0.940659,0.584654,0.258162,0.176249,0.130735,0.048000,0.279070,0.464062,0.023438,3.070312,3.046875,0.067483


In [16]:
y_train.head()

943     1
2486    0
517     1
778     1
1280    1
Name: label, dtype: int64

In [17]:
x_test.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
2534,0.195284,0.037663,0.195728,0.172961,0.218835,0.045874,1.730322,5.230177,0.883878,0.275524,0.173981,0.195284,0.156238,0.047105,0.279070,1.265625,0.140625,7.429688,7.289062,0.094094
1436,0.179490,0.060427,0.181524,0.133874,0.225089,0.091216,1.513972,5.995542,0.927290,0.447723,0.177439,0.179490,0.121193,0.047666,0.272727,0.690589,0.023438,4.546875,4.523438,0.113717
2568,0.221598,0.027525,0.220678,0.210593,0.236695,0.026102,3.449305,18.060482,0.796497,0.121375,0.219492,0.221598,0.203196,0.050104,0.266667,0.668750,0.210938,4.125000,3.914062,0.106897
3156,0.201806,0.036057,0.201622,0.178165,0.227872,0.049707,1.585353,4.945634,0.884731,0.227903,0.176117,0.201806,0.191704,0.032720,0.275862,0.593750,0.007812,5.921875,5.914062,0.124383
1473,0.211044,0.049072,0.215786,0.173499,0.254419,0.080920,1.398761,4.859347,0.913982,0.221827,0.264686,0.211044,0.110204,0.017003,0.275862,0.710677,0.007812,5.476562,5.468750,0.123399


In [18]:
y_test.head()

2534    0
1436    1
2568    0
3156    0
1473    1
Name: label, dtype: int64

## Decission_Tree

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
dtc_clf = DecisionTreeClassifier()
dtc_clf.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [53]:
dtc_clf_acc = cross_val_score(dtc_clf, x_train, y_train, cv=2, scoring="accuracy",  n_jobs=-1)
dtc_proba = cross_val_predict(dtc_clf, x_train, y_train, cv=2, method='predict_proba')
dtc_clf_scores = dtc_proba[:, 1]

In [54]:
dt_ypre=dtc_clf.predict(x_test)

In [55]:
print("\nAccuracy score: %f" %(accuracy_score(y_test,dt_ypre) * 100))
print("Recall score : %f" %(recall_score(y_test, dt_ypre) * 100))
print("ROC score : %f\n" %(roc_auc_score(y_test, dt_ypre) * 100))
print(confusion_matrix(y_test, dt_ypre)) 
print("F1 score : %f\n" %(f1_score(y_test, dt_ypre, zero_division=1)*100))


Accuracy score: 96.214511
Recall score : 96.074380
ROC score : 96.217062

[[450  17]
 [ 19 465]]
F1 score : 96.273292



In [56]:
print(list(zip(y_test.values,dt_ypre))[0:10])

[(1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (1, 1), (0, 0), (0, 0)]


## LogisticRegression

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)
lr_acc = cross_val_score(lr, x_train, y_train, cv=3, scoring='accuracy', n_jobs=-1)
lr_proba = cross_val_predict(lr, x_train, y_train, cv=3, method='predict_proba')
lr_scores = lr_proba[:, 1]

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_ms

In [20]:
lr_ypre=lr.predict(x_test)

In [21]:
print(list(zip(y_test.values,lr_ypre))[0:10])

[(0, 0), (1, 1), (0, 0), (0, 0), (1, 1), (1, 1), (0, 1), (0, 0), (1, 1), (0, 0)]


In [25]:
print("\nAccuracy score: %f" %(accuracy_score(y_test,lr_ypre) * 100))
print("Recall score : %f" %(recall_score(y_test, lr_ypre) * 100))
print("ROC score : %f\n" %(roc_auc_score(y_test, lr_ypre) * 100))
print(confusion_matrix(y_test, lr_ypre)) 
print("F1 score : %f\n" %(f1_score(y_test, lr_ypre, zero_division=1)*100))
print("Cost : %f\n" %np.sqrt(((lr_ypre - y_test) ** 2).mean()))


Accuracy score: 89.379600
Recall score : 94.939271
ROC score : 89.154537

[[381  76]
 [ 25 469]]
F1 score : 90.279115

Cost : 0.325890



## Random_Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier
rmf = RandomForestClassifier(max_depth=4, random_state=0)
rmf_clf = rmf.fit(x_train, y_train)

In [57]:
rmf_clf_acc = cross_val_score(rmf_clf, x_train, y_train, cv=3, scoring="accuracy",  n_jobs=-1)
rmf_proba = cross_val_predict(rmf_clf, x_train, y_train, cv=3, method='predict_proba')
rmf_clf_scores = rmf_proba[:, 1]

In [58]:
RF_ypre=rmf_clf.predict(x_test)

In [59]:
print(list(zip(y_test.values,RF_ypre))[0:10])

[(1, 1), (0, 0), (1, 1), (0, 0), (1, 1), (1, 1), (0, 0), (1, 1), (0, 0), (1, 1)]


In [60]:
print("\nAccuracy score: %f" %(accuracy_score(y_test,RF_ypre) * 100))
print("Recall score : %f" %(recall_score(y_test, RF_ypre) * 100))
print("ROC score : %f\n" %(roc_auc_score(y_test, RF_ypre) * 100))
print(confusion_matrix(y_test, RF_ypre))
print("F1 score : %f\n" %(f1_score(y_test, RF_ypre, zero_division=1)*100))


Accuracy score: 96.424816
Recall score : 96.624473
ROC score : 96.425444

[[459  18]
 [ 16 458]]
F1 score : 96.421053



## MLP_CLF

In [155]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
#trainX,trainY,testX,testY = X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
x_t = np.array(trainX)
x_tes = np.array(testX)
y_t = np.array(trainY)
y_tes = np.array(testY)


In [156]:
mlp = MLPClassifier(hidden_layer_sizes=(50,3),max_iter=301)

y_t = y_t.reshape(-1, 1)
print(x_t.shape,y_t.shape)
mlp.fit(x_t,y_t)
print("Training set score: %f" % mlp.score(x_t,y_t))
print("Test set score: %f" % mlp.score(x_tes,y_tes))

(2217, 20) (2217, 1)


E:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training set score: 0.961660
Test set score: 0.968454


In [158]:
print(clf.coefs_[0])
print('-------------------------------------------------')
print(clf.coefs_[1])
print('-------------------------------------------------')
print(clf.coefs_[2])

[[-0.0813015   0.21226532 -0.49396274 -0.19337576 -0.3477595 ]
 [-0.39942497 -0.30832521 -0.15238865 -0.10108883  0.03760951]
 [-0.07917288  0.17777444 -0.29386365  0.3707801  -0.46475979]
 [ 0.16702337 -0.08414688  0.05388862 -0.35208487 -0.2972294 ]
 [ 0.29466829  0.45458637 -0.18768512  0.18877811  0.36685465]
 [ 0.38663399 -0.40766391 -0.4528957  -0.32342014  0.37000305]
 [-0.39353812 -0.13431082  0.38261475  0.03468103  0.16187202]
 [-0.18075703 -0.31269164 -0.24585475 -0.46506401  0.01804986]
 [ 0.47898409  0.22672677 -0.23413185  0.28465932 -0.39627621]
 [-0.05105371  0.39389721 -0.20966999 -0.20757926 -0.36544227]
 [-0.47092228  0.17186027 -0.28643876 -0.22941021 -0.00979713]
 [-0.43761355  0.06901241 -0.35031075  0.08779917  0.19407005]
 [-0.38963109 -0.08741508  0.18675985 -0.08385205 -0.44242199]
 [ 0.03517115  0.1597437   0.01372969  0.43567997  0.08446662]
 [ 0.39525154 -0.3601752  -0.35919386  0.3015685  -0.10253333]
 [-0.32788458  0.40120251 -0.16961643  0.2479539   0.21

In [162]:
predictions_MLP = mlp.predict(testX)
predictions_MLP

array([1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,

In [163]:
print("\nAccuracy score: %f" %(accuracy_score(testY,predictions_MLP) * 100))
print("Recall score : %f" %(recall_score(testY,predictions_MLP) * 100))
print("ROC score : %f\n" %(roc_auc_score(testY,predictions_MLP) * 100))
print(confusion_matrix(testY,predictions_MLP)) 
print("F1 score : %f\n" %(f1_score(testY,predictions_MLP, zero_division=1)*100))


Accuracy score: 96.845426
Recall score : 100.000000
ROC score : 97.093023

[[162  10]
 [  0 145]]
F1 score : 96.666667

